<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/elegiveis_cb_fees/elegiveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd

In [2]:
!pip install dask[dataframe] pyxlsb

In [3]:
#especificar o nome do ficheiro
resumo_r_nome="Resumo_Proativa_092023.xlsb"
resumo_r1_nome="Resumo_Proativa_102023.xlsb"

In [4]:
#importar ficheiros

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()

Saving Resumo_Proativa_092023.xlsb to Resumo_Proativa_092023.xlsb


Saving Resumo_Proativa_102023.xlsb to Resumo_Proativa_102023.xlsb


In [5]:
#criar resumo mês n
resumo_r=pd.read_excel(resumo_r_nome, sheet_name="Auto", engine="pyxlsb")

In [6]:
#criar resumo mês n-3

resumo_r1=pd.read_excel(resumo_r1_nome, sheet_name="Auto", engine="pyxlsb")

In [20]:
#renomear colunas da bd consoante reactiva ou proactiva

resumo_n=resumo_r.copy()
resumo_n1=resumo_r1.copy()

def renomear_cols (df):
  lista=df.columns.tolist()
  lista2=[]
  for i in lista:
    if i.find("RRR")!=-1:
      lista2.append(i.replace("RRR", "").strip())
    elif i.find("PRR")!=-1:
      lista2.append(i.replace("PRR", "").strip())
    else:
      lista2.append(i)
  return lista2

resumo_n.columns=renomear_cols(resumo_n)
resumo_n1.columns=renomear_cols(resumo_n1)

In [30]:
#extrair linhas para avaliar se são elegíveis para cb - event type name="SERVICE REQUEST" e "ACTIVAÇÃO DE SERVIÇO"

def event_sr (df, ciclo, evento=["SERVICE REQUEST", "ACTIVAÇÃO DE SERVIÇO"]):
  if ciclo=="RRR":
    lista_incentivo=["RRR_CHARGEBACK NOS", 'RRR_PREVISÃO DE CHARGEBACK NOS']
    for i in evento:
      if i=="SERVICE REQUEST":
        df_sr=df.loc[(~df["Incentive Type Name"].isin(lista_incentivo)) & (df["Event Type Name"]==i), :]
      elif i=="ACTIVAÇÃO DE SERVIÇO":
        df_ativacao=df.loc[(~df["Incentive Type Name"].isin(lista_incentivo)) & (df["Event Type Name"]==i), :]
  else:
    lista_incentivo=["RPR_CHARGEBACK NOS", 'RPR_PREVISÃO DE CHARGEBACK NOS']
    for i in evento:
      if i=="SERVICE REQUEST":
        df_sr=df.loc[(~df["Incentive Type Name"].isin(lista_incentivo)) & (df["Event Type Name"]==i), :]
      elif i=="ACTIVAÇÃO DE SERVIÇO":
        df_ativacao=df.loc[(df["Incentive Type Name"].str.find("UPFRONT")!=-1) & (df["Event Type Name"]==i), :]


  return df_sr, df_ativacao


df_sr, df_ativacao=event_sr(resumo_n, "PRR")



In [23]:
#extrair linhas para avaliar se são elegíveis para cb - Chargeback Reason=Activação Negativa + portabilidades

def event_cb (df, df1, col=["Chargeback Reason", "Event Type Name"], str="Activação Negativa", str2=["EVENTO GENÉRICO DE SERVIÇO", "PORTABILIDADE DE SERVIÇO"]):
  for i in col:
    if i=="Chargeback Reason":
      df_cb=df1.loc[df1[i]==str, :]
    else:
      df_portabilidade=df.loc[(df[i].isin(str2)) & (df["Incentive Type Name"].str.find("PORTABILIDADE")!=-1), :]
  return df_cb, df_portabilidade

df_cb, df_portabilidade=event_cb(resumo_n, resumo_n1)

In [35]:
#extrair linhas para actividades extra: RR PAGAMENTO DE FACTURA, ACORDO DE PAGAMENTO

def extra (df, col="Incentive Type Name", col2="Event Type Name"):
  lista_evento=["RR PAGAMENTO DE FACTURA", "ACORDO DE PAGAMENTO"]
  lista_incentivo=["RPR_CHARGEBACK NOS", "RPR_PREVISÃO DE CHARGEBACK NOS"]
  df_extra=df.loc[(df[col2].isin(lista_evento)) & ~(df[col].isin(lista_incentivo)), :]
  return df_extra

df_extra=extra(resumo_n)

In [36]:
df_extra.shape

(7727, 100)

In [38]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.1 MB/s eta 0:00:00


In [39]:
#add different dataframes to the same excel file

excel_file_path = 'multi_sheet_excel.xlsx'
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    df_sr.to_excel(writer, sheet_name='SR', index=False)
    df_ativacao.to_excel(writer, sheet_name='Ativação', index=False)
    df_cb.to_excel(writer, sheet_name='GAL-1', index=False)
    df_portabilidade.to_excel(writer, sheet_name='Portabilidades', index=False)
    df_extra.to_excel(writer, sheet_name='Extras', index=False)


In [40]:
files.download('multi_sheet_excel.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>